# CKKS: Approximate Arithmetic

**Module 11** | 11-homomorphic-encryption

*Encoding reals, rescaling*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **ckks: approximate arithmetic**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [The BFV Scheme](11d-bfv-scheme.ipynb).
- Concepts and notation introduced in the previous notebook.

## Encoding Real Numbers

In [ ]:
# CKKS encodes real/complex numbers into polynomials
# Encode: scale by Delta, round to integer polynomial
# Decode: divide by Delta
# TODO: implement CKKS encoding

## Rescaling

In [ ]:
# After multiplication, scale grows: Delta^2
# Rescale: divide ciphertext by Delta to get back to Delta
# This is the key innovation of CKKS
# TODO: demonstrate rescaling

## Approximate Computations

In [ ]:
# CKKS allows small errors in the result
# Perfect for ML inference, statistics, signal processing
# TODO: compute a simple function on encrypted reals

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **ckks: approximate arithmetic**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*